In [ ]:
# ===========================================================
# Install dependencies
# ===========================================================
!pip install git+https://github.com/facebookresearch/sam2.git
!pip install supervision diffusers transformers accelerate scipy safetensors
!pip install opencv-python matplotlib tqdm pandas
!pip install ultralytics



In [9]:
# ===========================================================
# SAM2 Segmentation with Throughput Measurement
# ===========================================================

import os
import cv2
import time
import numpy as np
from PIL import Image
from ultralytics import SAM
from google.colab import drive

# Paths
base_drive = "/content/drive/MyDrive/sam2"
input_folder ="/content/drive/MyDrive/sam2/images"
output_root ="/content/drive/MyDrive/sam2/segments"
os.makedirs(output_root, exist_ok=True)

# Load SAM2 model
sam_model = SAM("sam2.1_b.pt")

# Mask refinement
def refine_mask(mask):
    mask_uint8 = (mask > 0).astype(np.uint8) * 255
    num_labels, labels = cv2.connectedComponents(mask_uint8)
    refined_masks = []

    if num_labels <= 2:
        dist = cv2.distanceTransform(mask_uint8, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist, 0.4 * dist.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(mask_uint8, sure_fg)
        num_markers, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0
        img_color = np.stack([mask_uint8]*3, axis=-1)
        cv2.watershed(img_color, markers)
        for label in np.unique(markers):
            if label <= 1:
                continue
            refined = (markers == label).astype(np.uint8)
            if refined.sum() > 50:
                refined_masks.append(refined)
    else:
        for label in range(1, num_labels):
            refined = (labels == label).astype(np.uint8)
            if refined.sum() > 50:
                refined_masks.append(refined)

    return refined_masks

# ===========================
# Process images with TT calc
# ===========================

image_files = [
    f for f in os.listdir(input_folder)
    if f.lower().endswith(('.jpg', '.png', '.jpeg'))
]

start_time = time.time()
processed_images = 0

for filename in image_files:
    img_path = os.path.join(input_folder, filename)
    image_bgr = cv2.imread(img_path)
    if image_bgr is None:
        continue

    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    h, w = image_rgb.shape[:2]

    base_name = os.path.splitext(filename)[0]
    out_dir = os.path.join(output_root, base_name)
    os.makedirs(out_dir, exist_ok=True)

    obj_count = 0
    results = sam_model(image_rgb)
    if results[0].masks is None:
        print(f" No masks detected by SAM for {filename}")
        continue

    for mask in results[0].masks.data.cpu().numpy():
        refined_masks = refine_mask(mask)
        for refined in refined_masks:
            obj_count += 1
            mask_ref = (refined.astype(np.uint8)) * 255
            rgba = np.zeros((h, w, 4), dtype=np.uint8)
            rgba[..., :3] = image_rgb
            rgba[..., 3] = mask_ref

            coords = cv2.findNonZero(mask_ref)
            if coords is None:
                continue
            x, y, w_box, h_box = cv2.boundingRect(coords)
            cropped_obj = rgba[y:y+h_box, x:x+w_box]

            out_name = f"segment_{obj_count}.png"
            Image.fromarray(cropped_obj).save(os.path.join(out_dir, out_name))

    processed_images += 1
    print(f" {filename}: saved {obj_count} segments → {out_dir}")

end_time = time.time()
total_time = end_time - start_time
throughput = processed_images / total_time if total_time > 0 else 0

print("\n======================================")
print(f"Processed {processed_images} images in {total_time:.2f} seconds")
print(f"Throughput: {throughput:.2f} images per second")
print("======================================")



0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 15563.2ms
Speed: 89.0ms preprocess, 15563.2ms inference, 1.1ms postprocess per image at shape (1, 3, 1024, 1024)
 75.jpeg: saved 8 segments → /content/drive/MyDrive/sam2/segments/75

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 7836.4ms
Speed: 11.5ms preprocess, 7836.4ms inference, 1.4ms postprocess per image at shape (1, 3, 1024, 1024)
 42.jpeg: saved 18 segments → /content/drive/MyDrive/sam2/segments/42

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 7826.9ms
Speed: 7.5ms preprocess, 7826.9ms inference, 1.4ms postprocess per image at shape (1, 3, 1024, 1024)
 38.jpeg: saved 22 segments → /content/drive/MyDrive/sam2/segments/38

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1